In [40]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as Soup
import os

In [41]:
MAX_LEN = 150

#prompt format: "A sticker of {...}" - без точки на конце.


def preprocess(s):
    s = s.replace(r'\n', '').replace('\n', '').strip().capitalize()
    return s

def parse_chatgpt_chat(content): #тут могут быть ошибки.
    body = Soup(content, 'lxml').find('body')
    string = body.find_all('script')[1].text
    prompts = []
    parts_idx = [m.start() for m in re.finditer("parts", string)]
    for i in range(len(parts_idx) - 1):
        text = string[parts_idx[i]: parts_idx[i + 1]][7:]
        text = text[:text.find('},')]
        lst = list(re.findall(r"A sticker of .*?\\n", text, re.DOTALL))
        last = text[text.rfind('A sticker of'):].strip().capitalize()
        if last.count(r'\n') > 0:
            lst.append(last[:last.find(r'\n')])
        else:
            lst.append(last.replace('"]', ''))
        prompts.extend([i for i in [preprocess(q) for q in lst] if i.endswith('.') and i.count('.') == 1])

    return list(set(prompts))

In [42]:
def load_urls(filename):
    with open(filename, mode="r", encoding="UTF-8") as urls_file:
        result_urls = []
        for line in urls_file.readlines():
            result_urls.append(line.strip())
            
    print(result_urls)
    return result_urls

In [43]:
chat_directory_path = "html-chats/"

def load_all_html_chats():
    result_html_chats = []
    for filename in os.listdir(chat_directory_path):
        file_path = os.path.join(chat_directory_path, filename)
        result_html_chats.append(file_path)
    return result_html_chats


In [70]:
# html_chats = load_all_html_chats()
html_chats = ["html-chats/chat-5.html"]

generated_prompts = []

for filename in html_chats:
    print("Parsing prompts from: ", filename)
    with open(filename, mode="r", encoding="UTF-8") as html_chat:
        generated_prompts.extend(parse_chatgpt_chat(html_chat))

Parsing prompts from:  html-chats/chat-5.html


In [71]:
len(generated_prompts)

1146

In [45]:
def dump_prompts(generated_prompts, batch_id):
    
    def get_prompts_batch_name(batch_id):
        return "collected-prompts/prompts-batch-" + str(batch_id) + ".txt"
    
    with open(get_prompts_batch_name(batch_id), mode="w", encoding="UTF-8") as prompts_batch_file:
        for prompt in generated_prompts:
            prompts_batch_file.write(prompt + "\n")

In [72]:
batch_id = 4

dump_prompts(generated_prompts, batch_id)